In [33]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1. Requirements


In [ ]:
!unzip /content/drive/MyDrive/HIISS/Causal-Unsupervised-Segmentation.zip &> /dev/null
!cp -r /content/content/Causal-Unsupervised-Segmentation /content
!rm -r /content/content

In [ ]:
!pip install -r /content/Causal-Unsupervised-Segmentation/requirements.txt &> /dev/null
!pip install timm==0.9.5 &> /dev/null
!pip install git+https://github.com/lucasb-eyer/pydensecrf.git &> /dev/null

## 1.1 Get DINO checkpoint

In [ ]:
!mkdir checkpoint
!wget https://dl.fbaipublicfiles.com/dino/dino_deitsmall8_pretrain/dino_deitsmall8_pretrain.pth &> /dev/null
!mv /content/dino_deitsmall8_pretrain.pth /content/checkpoint/dino_vit_small_8.pth

# 2 Train with four classes

## 2.1 Load pirelli dataset

Choose between strided and not strided

In [ ]:
!unzip /content/drive/MyDrive/HIISS/datasets/Pirelli_4cat_strided.zip -d /content &> /dev/null

## 2.2 CAUSE Super Crop


In [51]:
!python ./Causal-Unsupervised-Segmentation/crop_dataset.py  --data_dir "Pirelli_4cat" --dataset "pirelli" --crop_type "super"

Device = cuda:0
100% 130/130 [00:28<00:00,  4.61it/s]


## 2.3 Train the mediator
Namely get the clusterbook

In [62]:
!python ./Causal-Unsupervised-Segmentation/train_mediator.py \
                          --data_dir "Pirelli_4cat" --dataset "pirelli" \
                          --ckpt "/content/checkpoint/dino_vit_small_8.pth" \
                          --gpu 0 --epoch 3

------------------Configurations------------------
epoch: 3
distributed: True
load_segment: False
load_cluster: False
train_resolution: 320
test_resolution: 320
batch_size: 16
num_workers: 0
data_dir: Pirelli_4cat
dataset: pirelli
ckpt: /content/checkpoint/dino_vit_small_8.pth
gpu: 0
port: 12355
grid: True
num_codebook: 2048
reduced_dim: 90
projection_dim: 2048
dim: 384
-------------------------------------------------
_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=[])
No File Exists!!
-------------TRAIN EPOCH: 1-------------
[Train]: 100% 407/407 [01:56<00:00,  3.49it/s]
-------------TRAIN EPOCH: 2-------------
[Train]: 100% 407/407 [01:55<00:00,  3.53it/s]
-------------TRAIN EPOCH: 3-------------
[Train]: 100% 407/407 [01:55<00:00,  3.53it/s]


## 2.4 Train the transformer segmentator.

The clusterbook is fixed. Now, we train the transformer segmentator

In [63]:
!python ./Causal-Unsupervised-Segmentation/train_front_door_tr.py \
          --data_dir "Pirelli_4cat" --dataset "pirelli" \
          --ckpt "/content/checkpoint/dino_vit_small_8.pth" \
          --gpu 0 --epoch 5 --batch_size 16

------------------Configurations------------------
NAME_TAG: CAUSE-TR
data_dir: Pirelli_4cat
dataset: pirelli
ckpt: /content/checkpoint/dino_vit_small_8.pth
epoch: 5
distributed: False
load_segment: False
load_cluster: False
train_resolution: 320
test_resolution: 320
batch_size: 16
num_workers: 0
gpu: 0
port: 12355
grid: True
num_codebook: 2048
reduced_dim: 90
projection_dim: 2048
dim: 384
num_queries: 1600
-------------------------------------------------
_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=[])
4
Modularity CAUSE/pirelli/modularity/dino_vit_small_8/2048/modular.npy loaded
-------------TRAIN EPOCH: 1-------------
[Train] Loss: 7.17=6.84+0.33 ACC: 89.0%: 100% 407/407 [04:00<00:00,  1.69it/s]
-------------TEST EPOCH: 1-------------
[TEST] Acc (Linear): 93.0%: 100% 3/3 [00:01<00:00,  2.31it/s]
-----------------TEST Epoch 0: SAVING CHECKPOINT IN CAUSE/pirelli/dino_vit_small_8/2048/segment_tr.pth-----------------
-------------TRAIN EPOCH: 2----------

## 2.5 Fine tuning

Now, we fine tune the transformer and the clustebook. Twice following empirical results.

In [64]:
!python /content/Causal-Unsupervised-Segmentation/fine_tuning_tr.py \
          --data_dir "Pirelli_4cat" --dataset "pirelli" \
          --ckpt "/content/checkpoint/dino_vit_small_8.pth"  \
          --gpu 0 --epoch 3 --batch_size 16 --load_segment True \
          --load_cluster False

------------------Configurations------------------
NAME_TAG: CAUSE-TR
data_dir: Pirelli_4cat
dataset: pirelli
ckpt: /content/checkpoint/dino_vit_small_8.pth
epoch: 3
distributed: True
load_segment: True
load_cluster: False
train_resolution: 320
test_resolution: 320
batch_size: 16
num_workers: 0
gpu: 0
port: 12355
grid: True
num_codebook: 2048
reduced_dim: 90
projection_dim: 2048
dim: 384
num_queries: 1600
-------------------------------------------------
_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=[])
4
[Segment] CAUSE/pirelli/dino_vit_small_8/2048/segment_tr.pth loaded
Modularity CAUSE/pirelli/modularity/dino_vit_small_8/2048/modular.npy loaded
-------------TRAIN EPOCH: 1-------------
[Train] Loss: -0.00=0.10-0.10 ACC: 96.6%: 100% 407/407 [02:49<00:00,  2.41it/s]
-------------TEST EPOCH: 1-------------
[TEST] Acc (Linear): 97.9% | [mIoU]: 47.3, [mAP]: 55.5, [Acc]: 93.8, : 100% 3/3 [00:01<00:00,  2.17it/s]
-----------------TEST Epoch 0: SAVING CHECKPOIN

### Phase 2 retrain using the old cluster book.

In [65]:
!python /content/Causal-Unsupervised-Segmentation/fine_tuning_tr.py \
          --data_dir "Pirelli_4cat" --dataset "pirelli" \
          --ckpt "/content/checkpoint/dino_vit_small_8.pth"  \
          --gpu 0 --epoch 1 --batch_size 16 --load_segment True \
          --load_cluster False

------------------Configurations------------------
NAME_TAG: CAUSE-TR
data_dir: Pirelli_4cat
dataset: pirelli
ckpt: /content/checkpoint/dino_vit_small_8.pth
epoch: 1
distributed: True
load_segment: True
load_cluster: False
train_resolution: 320
test_resolution: 320
batch_size: 16
num_workers: 0
gpu: 0
port: 12355
grid: True
num_codebook: 2048
reduced_dim: 90
projection_dim: 2048
dim: 384
num_queries: 1600
-------------------------------------------------
_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=[])
4
[Segment] CAUSE/pirelli/dino_vit_small_8/2048/segment_tr.pth loaded
Modularity CAUSE/pirelli/modularity/dino_vit_small_8/2048/modular.npy loaded
-------------TRAIN EPOCH: 1-------------
[Train] Loss: -0.07=0.05-0.12 ACC: 97.9%: 100% 407/407 [02:49<00:00,  2.40it/s]
-------------TEST EPOCH: 1-------------
[TEST] Acc (Linear): 98.1% | [mIoU]: 67.0, [mAP]: 70.0, [Acc]: 97.9, : 100% 3/3 [00:01<00:00,  2.13it/s]
-----------------TEST Epoch 0: SAVING CHECKPOIN

## 2.6 Test the model

In [66]:
!python /content/Causal-Unsupervised-Segmentation/test_tr.py \
        --data_dir "Pirelli_4cat" --dataset "pirelli" \
        --ckpt "/content/checkpoint/dino_vit_small_8.pth" \
        --load_segment True --load_cluster True

------------------Configurations------------------
NAME_TAG: CAUSE-TR
data_dir: Pirelli_4cat
dataset: pirelli
port: 12355
ckpt: /content/checkpoint/dino_vit_small_8.pth
distributed: False
load_segment: True
load_cluster: True
train_resolution: 320
test_resolution: 320
batch_size: 16
num_workers: 0
gpu: 0
num_codebook: 2048
just_imgs: False
reduced_dim: 90
projection_dim: 2048
dim: 384
num_queries: 1600
-------------------------------------------------
_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=[])
4
[Segment] CAUSE/pirelli/dino_vit_small_8/2048/segment_tr.pth loaded
[Cluster] CAUSE/pirelli/dino_vit_small_8/2048/cluster_tr.pth loaded
Modularity CAUSE/pirelli/modularity/dino_vit_small_8/2048/modular.npy loaded
# of Parameters: 3.59(M)
Test without CRF
[TEST] Acc (Linear): 98.1% | [mIoU]: 67.0, [mAP]: 70.0, [Acc]: 97.9, : 100% 3/3 [00:02<00:00,  1.39it/s]
OrderedDict([('mIoU', tensor([97.6686, 96.7288, 73.6104,  0.0000], device='cuda:0')), ('mAP', tensor(

## 2.7 Save Results and checkpoints

In [67]:
!zip -r results_only_sup_4classes_strided.zip ./results &> /dev/null

In [68]:
!zip -r /content/CAUSE_only_sup_4classes_strided.zip ./CAUSE &> /dev/null

In [69]:
!mv /content/results_only_sup_4classes_strided.zip /content/drive/MyDrive/HIISS/results
!mv /content/CAUSE_only_sup_4classes_strided.zip /content/drive/MyDrive/HIISS/checkpoints

# 3 Train with three classes




In [ ]:
!rm -r /content/CAUSE # If retrain all

## 3.1 Load Pirelli 3 classes dataset

In [ ]:
!unzip /content/drive/MyDrive/HIISS/datasets/Pirelli_3cat_no_stride.zip -d /content &> /dev/null

## 3.2 Crop dataset

In [ ]:
!python ./Causal-Unsupervised-Segmentation/crop_dataset.py --data_dir "Pirelli_3cat" --dataset "pirelli" --crop_type "super"

Device = cuda:0
100% 130/130 [00:27<00:00,  4.69it/s]


## 3.3 Train Mediator

In [ ]:
!python ./Causal-Unsupervised-Segmentation/train_mediator.py \
                          --data_dir "Pirelli_3cat" --dataset "pirelli" \
                          --ckpt "/content/checkpoint/dino_vit_small_8.pth" \
                          --gpu 0 --epoch 3

------------------Configurations------------------
epoch: 3
distributed: True
load_segment: False
load_cluster: False
train_resolution: 320
test_resolution: 320
batch_size: 16
num_workers: 0
data_dir: Pirelli_3cat
dataset: pirelli
ckpt: /content/checkpoint/dino_vit_small_8.pth
gpu: 0
port: 12355
grid: True
num_codebook: 2048
reduced_dim: 90
projection_dim: 2048
dim: 384
-------------------------------------------------
_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=[])
No File Exists!!
-------------TRAIN EPOCH: 1-------------
[Train]: 100% 407/407 [01:55<00:00,  3.53it/s]
-------------TRAIN EPOCH: 2-------------
[Train]: 100% 407/407 [01:54<00:00,  3.55it/s]
-------------TRAIN EPOCH: 3-------------
[Train]: 100% 407/407 [01:54<00:00,  3.54it/s]


## 3.4 Train Transformer

In [ ]:
!python ./Causal-Unsupervised-Segmentation/train_front_door_tr.py \
          --data_dir "Pirelli_3cat" --dataset "pirelli" \
          --ckpt "/content/checkpoint/dino_vit_small_8.pth" \
          --gpu 0 --epoch 4 --batch_size 16

------------------Configurations------------------
NAME_TAG: CAUSE-TR
data_dir: Pirelli_3cat
dataset: pirelli
ckpt: /content/checkpoint/dino_vit_small_8.pth
epoch: 4
distributed: False
load_segment: False
load_cluster: False
train_resolution: 320
test_resolution: 320
batch_size: 16
num_workers: 0
gpu: 0
port: 12355
grid: True
num_codebook: 2048
reduced_dim: 90
projection_dim: 2048
dim: 384
num_queries: 1600
-------------------------------------------------
_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=[])
3
Modularity CAUSE/pirelli/modularity/dino_vit_small_8/2048/modular.npy loaded
-------------TRAIN EPOCH: 1-------------
[Train] Loss: 7.19=6.84+0.35 ACC: 87.1%: 100% 407/407 [03:57<00:00,  1.71it/s]
-------------TEST EPOCH: 1-------------
[TEST] Acc (Linear): 91.8%: 100% 3/3 [00:01<00:00,  2.37it/s]
-----------------TEST Epoch 0: SAVING CHECKPOINT IN CAUSE/pirelli/dino_vit_small_8/2048/segment_tr.pth-----------------
-------------TRAIN EPOCH: 2----------

## 3.5 Fine-tuning

In [ ]:
!python /content/Causal-Unsupervised-Segmentation/fine_tuning_tr.py \
          --data_dir "Pirelli_3cat" --dataset "pirelli" \
          --ckpt "/content/checkpoint/dino_vit_small_8.pth"  \
          --gpu 0 --epoch 2 --batch_size 16 --load_segment True \
          --load_cluster False

------------------Configurations------------------
NAME_TAG: CAUSE-TR
data_dir: Pirelli_3cat
dataset: pirelli
ckpt: /content/checkpoint/dino_vit_small_8.pth
epoch: 2
distributed: True
load_segment: True
load_cluster: False
train_resolution: 320
test_resolution: 320
batch_size: 16
num_workers: 0
gpu: 0
port: 12355
grid: True
num_codebook: 2048
reduced_dim: 90
projection_dim: 2048
dim: 384
num_queries: 1600
-------------------------------------------------
_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=[])
3
[Segment] CAUSE/pirelli/dino_vit_small_8/2048/segment_tr.pth loaded
Modularity CAUSE/pirelli/modularity/dino_vit_small_8/2048/modular.npy loaded
-------------TRAIN EPOCH: 1-------------
[Train] Loss: -0.02=0.10-0.12 ACC: 96.4%: 100% 407/407 [02:48<00:00,  2.42it/s]
-------------TEST EPOCH: 1-------------
[TEST] Acc (Linear): 97.9% | [mIoU]: 87.1, [mAP]: 92.2, [Acc]: 97.3, : 100% 3/3 [00:01<00:00,  2.17it/s]
-----------------TEST Epoch 0: SAVING CHECKPOIN

## 3.6 Retrain with the old clusterbook

In [ ]:
!python /content/Causal-Unsupervised-Segmentation/fine_tuning_tr.py \
          --data_dir "Pirelli_3cat" --dataset "pirelli" \
          --ckpt "/content/checkpoint/dino_vit_small_8.pth"  \
          --gpu 0 --epoch 1 --batch_size 16 --load_segment True \
          --load_cluster False

------------------Configurations------------------
NAME_TAG: CAUSE-TR
data_dir: Pirelli_3cat
dataset: pirelli
ckpt: /content/checkpoint/dino_vit_small_8.pth
epoch: 1
distributed: True
load_segment: True
load_cluster: False
train_resolution: 320
test_resolution: 320
batch_size: 16
num_workers: 0
gpu: 0
port: 12355
grid: True
num_codebook: 2048
reduced_dim: 90
projection_dim: 2048
dim: 384
num_queries: 1600
-------------------------------------------------
_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=[])
3
[Segment] CAUSE/pirelli/dino_vit_small_8/2048/segment_tr.pth loaded
Modularity CAUSE/pirelli/modularity/dino_vit_small_8/2048/modular.npy loaded
-------------TRAIN EPOCH: 1-------------
[Train] Loss: -0.06=0.05-0.11 ACC: 97.9%: 100% 407/407 [02:48<00:00,  2.41it/s]
-------------TEST EPOCH: 1-------------
[TEST] Acc (Linear): 98.0% | [mIoU]: 88.9, [mAP]: 91.6, [Acc]: 97.7, : 100% 3/3 [00:01<00:00,  1.92it/s]
-----------------TEST Epoch 0: SAVING CHECKPOIN

# Test Model

In [ ]:
!python /content/Causal-Unsupervised-Segmentation/test_tr.py \
        --data_dir "Pirelli_3cat" --dataset "pirelli" \
        --ckpt "/content/checkpoint/dino_vit_small_8.pth" \
        --load_segment True --load_cluster True

------------------Configurations------------------
NAME_TAG: CAUSE-TR
data_dir: Pirelli_3cat
dataset: pirelli
port: 12355
ckpt: /content/checkpoint/dino_vit_small_8.pth
distributed: False
load_segment: True
load_cluster: True
train_resolution: 320
test_resolution: 320
batch_size: 16
num_workers: 0
gpu: 0
num_codebook: 2048
just_imgs: False
reduced_dim: 90
projection_dim: 2048
dim: 384
num_queries: 1600
-------------------------------------------------
_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=[])
3
[Segment] CAUSE/pirelli/dino_vit_small_8/2048/segment_tr.pth loaded
[Cluster] CAUSE/pirelli/dino_vit_small_8/2048/cluster_tr.pth loaded
Modularity CAUSE/pirelli/modularity/dino_vit_small_8/2048/modular.npy loaded
# of Parameters: 3.59(M)
Test without CRF
[TEST] Acc (Linear): 98.0% | [mIoU]: 88.9, [mAP]: 91.6, [Acc]: 97.7, : 100% 3/3 [00:02<00:00,  1.27it/s]
OrderedDict([('mIoU', tensor([97.3722, 96.2258, 73.0141], device='cuda:0')), ('mAP', tensor([99.5882,

# Save Results and checkpoints

In [ ]:
!zip -r results_only_sup_3classes_no_stride.zip ./results &> /dev/null

In [ ]:
!zip -r /content/CAUSE_only_sup_3classes_no_stride.zip ./CAUSE &> /dev/null

In [ ]:
!mv /content/CAUSE_only_sup_3classes_no_stride.zip /content/drive/MyDrive/HIISS/checkpoints
!mv results_only_sup_3classes_no_stride.zip /content/drive/MyDrive/HIISS/results

# Predict

In [40]:
!rm -r /content/Pirelli_unsupervised

In [ ]:
!unzip /content/drive/MyDrive/HIISS/datasets/Pirelli_unsupervised_no_stride.zip -d /content &> /dev/null

In [45]:
!python ./Causal-Unsupervised-Segmentation/crop_dataset.py  --data_dir "Pirelli_unsupervised" --dataset "pirelli" --crop_type "super"

Device = cuda:0
100% 1335/1335 [11:00<00:00,  2.02it/s]


In [46]:
!python /content/Causal-Unsupervised-Segmentation/test_tr.py \
        --data_dir "Pirelli_unsupervised" --dataset "pirelli" \
        --ckpt "/content/checkpoint/dino_vit_small_8.pth" \
        --load_segment True --load_cluster True --just_imgs True

------------------Configurations------------------
NAME_TAG: CAUSE-TR
data_dir: Pirelli_unsupervised
dataset: pirelli
port: 12355
ckpt: /content/checkpoint/dino_vit_small_8.pth
distributed: False
load_segment: True
load_cluster: True
train_resolution: 320
test_resolution: 320
batch_size: 16
num_workers: 0
gpu: 0
num_codebook: 2048
just_imgs: True
reduced_dim: 90
projection_dim: 2048
dim: 384
num_queries: 1600
-------------------------------------------------
_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=[])
3
[Segment] CAUSE/pirelli/dino_vit_small_8/2048/segment_tr.pth loaded
[Cluster] CAUSE/pirelli/dino_vit_small_8/2048/cluster_tr.pth loaded
Modularity CAUSE/pirelli/modularity/dino_vit_small_8/2048/modular.npy loaded
# of Parameters: 3.59(M)
Test
100% 8/8 [02:27<00:00, 18.47s/it]


In [47]:
!zip -r results_only_sup_all_imgs.zip ./results &> /dev/null

In [48]:
!mv results_only_sup_all_imgs.zip /content/drive/MyDrive/HIISS/results